Coarsen a density grid and source file. 

In [27]:
import sys
print(sys.path)
import os

sys.path.insert(0, os.path.abspath('/home/iti20/software/pyC2Ray/pyc2ray-env/bin'))
sys.path.insert(0, os.path.abspath('/home/iti20/software/pyC2Ray/pyc2ray-env/lib/python3.12/site-packages/tools21cm-2.2.10-py3.12-linux-x86_64.egg'))


['/home/iti20/software/pyC2Ray/pyc2ray-env/lib/python3.12/site-packages/tools21cm-2.2.10-py3.12-linux-x86_64.egg', '/home/iti20/software/pyC2Ray/pyc2ray-env/bin', '/home/iti20/software/pyC2Ray/pyc2ray-env/lib/python3.12/site-packages/tools21cm-2.2.10-py3.12-linux-x86_64.egg', '/home/iti20/software/pyC2Ray/pyc2ray-env/bin', '/home/iti20/codes/pyC2Ray/docs/examples', '/home/iti20/software/tools21cm', '/home/iti20/software/pyC2Ray', '/home/iti20/c2raytools', '/home/iti20', '/home/iti20/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/opt/rocm-6.3.3/lib']


In [28]:
import pyc2ray as pc2r
import tools21cm as t2c
import numpy as np, matplotlib.pyplot as plt
import bisect

from matplotlib.colors import LogNorm

### Reading simulation data
Here we read the ionisation fraction data cube produced with the [pyC2Ray](https://github.com/cosmic-reionization/pyC2Ray) code. For the density field, we will consider the gridded density field created by an N-body code, [CubeP3M](https://github.com/jharno/cubep3m), which were used by pyC2Ray code as input.

In [29]:
path_to_densityfiles = '/media/iti20/4store_toshiba/244Mpc_WMAP3+/coarser_data_SPH_4000_244Mpc/nc250/'
zs_set=t2c.usefuls.get_zs_list(path_to_densityfiles, file_type='/*n_all.dat')

print(zs_set)

coarsening_fraction = 2  # by what factor do we want to coarsen the grid?
                       # make sure the original grid size is divisible by this factor

[ 5.75   6.     6.056  6.113  6.172  6.231  6.292  6.354  6.418  6.483
  6.549  6.617  6.686  6.757  6.83   6.905  6.981  7.059  7.139  7.221
  7.305  7.391  7.48   7.57   7.664  7.76   7.859  7.96   8.064  8.172
  8.283  8.397  8.515  8.636  8.762  8.892  9.026  9.164  9.308  9.457
  9.611  9.771  9.938 10.11  10.29  10.478 10.673 10.877 11.09  11.313
 11.546 11.791 12.048 12.318 12.603 12.903 13.221 13.557 13.914 14.294
 14.699 15.132 15.596 16.095 16.633 17.215 17.848 18.54  19.298 20.134
 21.062 22.1   23.268 24.597 26.124 27.9   30.   ]


In [ ]:
t2c.set_sim_constants(244) # This line is only useful while working with C2Ray simulations.

for z in zs_set:
    z_string = str(z)
    d_file = t2c.DensityFile(path_to_densityfiles+'%.3fn_all.dat' %z)
                             #% zs_set[1+bisect.bisect_left(zs_set, z+0.0001)])
    dtot_file = t2c.DensityFile(path_to_densityfiles+'%.3fntot_all.dat' %z)
    #z_float=float(z_string)
    dens  = d_file.raw_density
    dens_tot = dtot_file.raw_density
#print(np.shape(dens))

    grid_size=dens.shape[0]
#    print(grid_size)

#    dens_avg = np.sum(dens)/grid_size**3
#    print(dens_avg)

    coarse_grid_size = grid_size//coarsening_fraction

#    print(coarse_grid_size)
    if grid_size%coarsening_fraction!=0:
        print('this is not dividing evenly, fix it')
    coarse_density=np.zeros((coarse_grid_size,coarse_grid_size,coarse_grid_size), dtype='float32')
    coarse_totdensity=np.zeros((coarse_grid_size,coarse_grid_size,coarse_grid_size), dtype='float32')
    for i in range(coarse_grid_size):
        for j in range(coarse_grid_size):
            for k in range(coarse_grid_size):
                coarse_density[i,j,k] = dens[i*coarsening_fraction:(i+1)*coarsening_fraction-1,j*coarsening_fraction:(j+1)*coarsening_fraction-1,k*coarsening_fraction:(k+1)*coarsening_fraction-1].sum()
                coarse_totdensity[i,j,k] = dens_tot[i*coarsening_fraction:(i+1)*coarsening_fraction-1,j*coarsening_fraction:(j+1)*coarsening_fraction-1,k*coarsening_fraction:(k+1)*coarsening_fraction-1].sum()

    filename = ('./'+'%.3fncoarse_all.dat' %z)  
    filename_tot = ('./'+'%.3fntotcoarse_all.dat' %z)

    f = open(filename, 'wb')
    ftot = open(filename_tot, 'wb')
    

    mesh_x, mesh_y, mesh_z = coarse_density.shape[0],coarse_density.shape[1],coarse_density.shape[2]
    mesh_coarse=np.array([mesh_x,mesh_y,mesh_z],dtype='int32')

    #print(mesh_coarse)

    #swap 1st and 3rd axes to orient the array the same as before (why were they flipped??)
    coarse_density_swapped=np.swapaxes(coarse_density, 0, 2)
    coarse_totdensity_swapped=np.swapaxes(coarse_totdensity, 0, 2)
    
    mesh_coarse.tofile(f)
    coarse_density_swapped.tofile(f)

    mesh_coarse.tofile(ftot)
    coarse_totdensity_swapped.tofile(ftot)    
    
    f.close()
    ftot.close()


In [ ]:
path_to_sourcefiles = '/media/iti20/4store_toshiba/244Mpc_WMAP3+/sources/'

for z in zs_set:
    s_file = path_to_sourcefiles+'%.3f-coarsest_sources.dat' %z
    print(s_file)
    src = np.loadtxt(s_file, skiprows=1)
    print(src[0,:])
    #src_coarse==src
    src[:,0:3]=src[:,0:3]//coarsening_fraction+src[:0:3]%coarsening_fraction
    new_file = './'+'%.3f-coarsened_sources.dat' %z
    np.savetxt(new_file,src)